In [13]:
import json

import gc_utils
import h5py
import numpy as np


In [14]:
sim = "m12i"
sim_dir = "/Users/z5114326/Documents/simulations/"

proc_file = sim_dir + sim + "/" + sim + "_processed.hdf5"
proc_data = h5py.File(proc_file, "r")  # open processed data file

In [15]:
it_min = 0
it_max = 100

min_gc_num = 5

snap = 600

snap_id = gc_utils.snapshot_name(snap)

In [16]:
init_common_groups = set()
for it in range(it_min, it_max + 1):
    it_id = gc_utils.iteration_name(it)
    snap_data = proc_data[it_id]["snapshots"][snap_id]
    groups = np.abs(snap_data["group_id"][()])

    unique_groups, counts = np.unique(groups, return_counts=True)
    repeat_groups = unique_groups[counts >= min_gc_num]

    init_common_groups.update(repeat_groups)

init_common_groups = list(init_common_groups)
common_groups = init_common_groups

# common_groups = []
# for grp in init_common_groups:
#     grp_len = []
#     for it in range(it_min, it_max + 1):
#         it_id = gc_utils.iteration_name(it)
#         snap_data = proc_data[it_id]["snapshots"][snap_id]
#         groups = np.abs(snap_data["group_id"][()])

#         groups = groups[groups == grp]
#         if len(groups) > 2:
#             grp_len.append(1)

#     if len(grp_len) == it_max + 1:
#         common_groups.append(grp)

In [17]:
common_groups

[0, 8580896, 13687078, 4414957, 16199669, 1920378, 19898495]

In [18]:
halt = gc_utils.get_halo_tree(sim, sim_dir)

Retrieving Halo Tree.....................: 100%|████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.24s/it]


In [19]:
snap_order = []
for grp in common_groups:
    if grp == 0:
        snap_order.append(0)

    else:
        halt_idx = np.where(halt["tid"] == grp)[0][0]
        snap_order.append(halt["snapshot"][halt_idx])

sorted_groups = [grp for _, grp in sorted(zip(snap_order, common_groups))]

In [20]:
common_colors = [
    "red",
    "blue",
    "green",
    "cyan",
    "magenta",
    "orange",
    "purple",
    "pink",
    "brown",
    "gold",
    "lime",
    "navy",
    "olive",
    "teal",
    "maroon",
]

In [21]:
group_dict = {sim: {}}

for i, grp in enumerate(sorted_groups):
    group_dict[sim][str(grp)] = common_colors[i]

In [22]:
group_dict[sim]

{'0': 'red',
 '1920378': 'blue',
 '4414957': 'green',
 '8580896': 'cyan',
 '13687078': 'magenta',
 '16199669': 'orange',
 '19898495': 'purple'}

In [23]:
# Save the dictionary to a JSON file
save_file = sim_dir + sim + "/" + "gc_groups.json"
with open(save_file, "w") as json_file:
    json.dump(group_dict, json_file, indent=4)

In [24]:
proc_data.close()